In [14]:
import nltk
import numpy as np
import random
import string
#print("Imported all")

In [24]:
import os

path = r"C:\Users\User\ChatBot.txt.txt"
if os.path.exists(path):
    print(f"The file exists: {path}")
else:
    print(f"The file does not exist: {path}")

print(os.getcwd())


The file exists: C:\Users\User\ChatBot.txt.txt
C:\Users\User


In [58]:
#for single file
'''
f=open("C:/Users/User/ChatBot.txt.txt",'r',errors='ignore')
raw=f.read()
raw=raw.lower() 
nltk.download('punkt') #pre-trained tokenizer and sentence boundary detector
nltk.download('wordnet')  #lexical database for the English
sent_tokens=nltk.sent_tokenize(raw) #Raw data to sentences
word_tokens=nltk.word_tokenize(raw) #Raw data to words
#print("Done")
'''
# Ensure required NLTK resources are downloaded
nltk.download('punkt')  # Pre-trained tokenizer and sentence boundary detector
nltk.download('wordnet')  # Lexical database for the English language

def process_directory(directory_path):
    all_raw_text = ""
    
    # Iterate over each file in the specified directory
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)  # Construct full file path
        if os.path.isfile(file_path):  # Check if it's a file
            try:
                with open(file_path, 'r', errors='ignore') as f:
                    raw = f.read()
                    all_raw_text += raw + " "  # Append the content of the file to the combined text
            except Exception as e:
                print(f"Error reading file {file_path}: {e}")
        else:
            print(f"Not a file: {file_path}")
    
    # Convert the combined raw text to lowercase
    all_raw_text = all_raw_text.lower()
    
    # Tokenize the combined text into sentences and words
    sent_tokens = nltk.sent_tokenize(all_raw_text)  # Raw data to sentences
    word_tokens = nltk.word_tokenize(all_raw_text)  # Raw data to words
    
    return sent_tokens, word_tokens

# Specify the directory containing the files to be processed
directory_path = "C:/Users/User/ChatBotFiles"

# Process the directory and obtain tokens
sent_tokens, word_tokens = process_directory(directory_path)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [60]:
#checking if above is working fine
print(sent_tokens[:5])
print(word_tokens[:5])

['a chatbot (originally chatterbot)[1] is a software application or web interface that is designed to mimic human conversation through text or voice interactions.', '[2][3][4] modern chatbots are typically online and use generative artificial intelligence systems that are capable of maintaining a conversation with a user in natural language and simulating the way a human would behave as a conversational partner.', 'such chatbots often use deep learning and natural language processing, but simpler chatbots have existed for decades.', "since late 2022, the field has gained widespread attention due to the popularity of openai's chatgpt,[5][6] followed by alternatives such as microsoft's copilot and google's gemini.", '[7] such examples reflect the recent practice of basing such products upon broad foundational large language models, such as gpt-4 or the gemini language model, that get fine-tuned so as to target specific tasks or applications (i.e., simulating human conversation, in the ca

In [62]:
#Preprocessing and Handling Raw Text

# Initialize the WordNetLemmatizer from nltk for lemmatization
lemmer = nltk.stem.WordNetLemmatizer()

# Define a function to lemmatize tokens (words)
def LemTokens(tokens):
    # Apply lemmatization to each token (word) in the list and return the list of lemmatized tokens
    return [lemmer.lemmatize(token) for token in tokens]

# Create a dictionary to map each punctuation character to None for removal
remove_punc_dict = dict((ord(punct), None) for punct in string.punctuation)

# Define a function to normalize text by tokenizing, lowercasing, and removing punctuation
def LemNormalize(text):
    # Convert text to lowercase, remove punctuation, and tokenize
    # Then apply lemmatization to the tokens and return the normalized tokens
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punc_dict)))

'''
Lemmatization is the process of reducing a word to its base or dictionary form, called a lemma.
eg: word- flies then Lemma-fly
'''

'\nLemmatization is the process of reducing a word to its base or dictionary form, called a lemma.\neg: word- flies then Lemma-fly\n'

In [64]:
#Greetings
Greeting_inputs=("hello","hi","greetings","sup","morning","hey")
Greeting_responses=["hi","hey","nods","hi there","hello","Hi, morning","Welcome","Glad, you sent a message!"]

def greeting(sentence):
    for word in sentence.split():
        if word.lower() in Greeting_inputs:
            return random.choice(Greeting_responses)

In [66]:
#use TF-IDF- Term Frequency-Inverse Document Frequency
from sklearn.feature_extraction.text import TfidfVectorizer

#find similarty b/w user and raw document
from sklearn.metrics.pairwise import cosine_similarity

In [68]:
# Function to generate a response based on user input
def responses(user_response):
    robo_response = ''  # Initialize an empty string for the response
    
    # Append the user response to the list of sentence tokens
    sent_tokens.append(user_response)
    
    # Initialize TfidfVectorizer with the LemNormalize tokenizer and English stop words
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words="english")
    
    # Fit the TfidfVectorizer on the sentence tokens and transform them into TF-IDF matrices
    tfidf = TfidfVec.fit_transform(sent_tokens)
    
    # Compute cosine similarity between the last sentence (user response) and all sentences
    vals = cosine_similarity(tfidf[-1], tfidf)
    
    # Get the index of the sentence with the highest similarity (excluding the last added user response)
    idx = vals.argsort()[0][-2]
    
    # Flatten the similarity values array and sort it
    flat = vals.flatten()
    flat.sort()
    
    # Retrieve the second highest similarity score
    req_tfidf = flat[-2]
    
    # Check if the highest similarity score is zero
    if req_tfidf == 0:
        # If similarity is zero, append a default response indicating lack of understanding
        robo_response = robo_response + "I am sorry! I don't understand you."
    else:
        # Otherwise, append the most similar sentence from the sent_tokens to the response
        robo_response = robo_response + sent_tokens[idx]
    
    # Return the generated response
    return robo_response


In [70]:
#Conversation b/w user and bot
flag= True
print("ChatBot:My Name is ChatBot1. I can help you with India, Space Flights and ChatBot, If you want to quit type Bye !!")
while(flag==True):
    user_response=input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=="thanks" or user_response =="thank you"):
            flag=False
            print("ChatBot:You are Welcome!")
        else:
            if(greeting(user_response)!=None):
                print("ChatBot:"+greeting(user_response))
            else:
                print("ChatBot:",end="")
                print(responses(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ChatBot:Thanks for converation, Bye!")


ChatBot:My Name is ChatBot1. I can help you with India, Space Flights and ChatBot, If you want to quit type Bye !!


 what is india


ChatBot:[117] the sultanate was to control much of north india and to make many forays into south india.


 what is space 


ChatBot:objectives for space missions may include space exploration, space research, and national firsts in spaceflight.


...

ChatBot:there were two prime ministers during this period; v.p.


 what are biodiversity


ChatBot:[69] india's land is megadiverse, with four biodiversity hotspots.


 bye


ChatBot:Thanks for converation, Bye!
